In [1]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import PyPDFLoader
from langchain.chains.question_answering import load_qa_chain
from langchain_community.document_loaders import TextLoader
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain
from langchain_community.vectorstores import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter
import os

In [2]:
api_key = os.environ.get("OPENAI_API_KEY")

In [3]:
# Load the embedding and LLM model
embeddings_model = OpenAIEmbeddings()
llm = ChatOpenAI(model_name = "gpt-4o", max_tokens = 1000)

/Users/rianrachmanto/miniforge3/envs/mlp/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(
/Users/rianrachmanto/miniforge3/envs/mlp/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [4]:
# Load data
loader = TextLoader("/Users/rianrachmanto/miniforge3/project/RAG_Drill_Report/data/well_remark.txt")
docs = loader.load()

In [5]:
text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)

In [6]:
db = Chroma.from_documents(documents, embedding = embeddings_model, persist_directory="/Users/rianrachmanto/miniforge3/project/RAG_Drill_Report/test_index")
db.persist()

/Users/rianrachmanto/miniforge3/envs/mlp/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  warn_deprecated(


In [7]:
retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": 4})

In [8]:
chain = load_qa_chain(llm, chain_type="stuff")

/Users/rianrachmanto/miniforge3/envs/mlp/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: This class is deprecated. See the following migration guides for replacements based on `chain_type`:
stuff: https://python.langchain.com/v0.2/docs/versions/migrating_chains/stuff_docs_chain
map_reduce: https://python.langchain.com/v0.2/docs/versions/migrating_chains/map_reduce_chain
refine: https://python.langchain.com/v0.2/docs/versions/migrating_chains/refine_chain
map_rerank: https://python.langchain.com/v0.2/docs/versions/migrating_chains/map_rerank_docs_chain

See also guides on retrieval and question-answering here: https://python.langchain.com/v0.2/docs/how_to/#qa-with-rag
  warn_deprecated(


In [86]:
# # A utility function for answer generation
# def ask(question):
#     context = retriever.get_relevant_documents(question)
#     answer = (chain({"input_documents": context,"question": question}, return_only_outputs=True))['output_text']
#     return answer


In [9]:
# A utility function for answer generation
def ask(question):
    # Define the prompt template
    prompt_template = """Answer the following question based only on the provided context.
    Please provide with detail answers. 
    If you don't know the answer, just say you don't know. Don't try to make up an answer.
    
    Context:
    {context}
    
    Question: {question}
    
    Answer: Let's approach this step by step:"""
    
    # Retrieve relevant documents
    context = retriever.get_relevant_documents(question)
    
    # Format the prompt with the context and question
    prompt = prompt_template.format(context=context, question=question)
    
    # Generate the answer using the prompt
    answer = (chain({"input_documents": context, "question": prompt}, return_only_outputs=True))['output_text']
    
    return answer

In [10]:
user_question = "List all the date of setting plug"
answer = ask(user_question)
print("Answer:", answer)

/Users/rianrachmanto/miniforge3/envs/mlp/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use invoke instead.
  warn_deprecated(
/Users/rianrachmanto/miniforge3/envs/mlp/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  warn_deprecated(


Answer: Based on the provided context, the dates on which plugs were set are as follows:

1. **01-Feb-2014**: The context mentions setting a PX Plug and Prong on the surface.
2. **11-Jan-2013**: A PX Plug body and prong were set inside during this date's operations.

These are the only dates mentioned in the context where plugs were set.
